In [1]:
%load_ext autoreload
%autoreload 2

import os, sys
root_dir = os.path.dirname(os.path.abspath(''))
if not root_dir in sys.path: sys.path.append(root_dir)

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from collections import Counter
import h5py
import scipy.io as sio

%matplotlib nbagg

In [3]:
from scipy.special import binom as binom_fun
from scipy.special import factorial as sp_factorial
from scipy.integrate import quad

import theano
import theano.tensor as T
from theano.graph.op import Op
from theano.graph.basic import Apply
from theano.compile.io import In

import pymc3 as pm
import arviz as az

In [4]:
from inference import *
from DM_theory import *
from empirical.readData import *
from empirical.model import *

In [5]:
class theano_binomial(Op):
    
    __props__ = ()
    itypes = [T.dscalar,T.dvector]
    otypes = [T.dvector]

    def perform(self,node,inputs,output_storage):
        # actually compute the binomial
        n,k = inputs
        output_storage[0][0] = np.array(binom_fun(n,k),dtype='float64')

In [6]:
#from DM_theory import *
def p_nu(nu,gamma,delta,nu_max):
    
    #try:
    #    nu = nu[np.newaxis,:]
    #except:
    #    print('no cast')
    return gamma / ( nu_max * T.sqrt( -np.pi * T.log( nu / nu_max ) ) ) * \
        T.exp( - delta**2/2.) * ( nu / nu_max )**(gamma**2 - 1) * \
        T.cosh( gamma * delta * T.sqrt( -2 * T.log( nu / nu_max) ) )

def poisson_spikes(nu,N,T_total):
    #T.prod(T.arange(N+1)[1:],axis=0,no_zeros_in_input=True)
    #try:
    #    nu = nu[np.newaxis,:]
    #except:
    #    print('no cast')
    return (nu*T_total)**N / T.gamma(N+1) * T.exp(-nu*T_total)

In [7]:
theano.config.optimizer = 'fast_run'
theano.config.exception_verbosity = 'high'
theano.config.on_unused_input = 'warn'
theano.config.mode = 'FAST_RUN'

In [10]:
#from empirical import *

class Inference:

    dataLoaded = False
    paras = {}

    def __init__(self):

        os.environ['MKL_NUM_THREADS'] = '1'
        os.environ['OPENBLAS_NUM_THREADS'] = '1'

    def set_model(self, func, paras=None):
        assert self.dataLoaded, "Data is not yet loaded, but required for setting the model parameters! (using 'load_data')"

        if callable(func):
            logp = func
        elif func=='selfcon':

            if not paras:
                paras = {
                    'gamma':{'mu':1.5, 'sigma':1.0, 'sigma_animal':1.0, 'prior':'Normal'},
                    'delta':{'mu':4., 'sigma':2., 'sigma_animal':1.0, 'prior':'Normal'},
                    'nu_max':{'mu':60., 'sigma':20., 'sigma_animal':5.0, 'prior':'Normal'}
                }
            assert all(key in paras for key in ("gamma","delta","nu_max")), "Please provide all necessary parameters!"
            def logp(n,k_with_N_AP,gamma,delta,nu_max):
                
                print('define logp from counts')
                
                T_total = theano.shared(10.)                     # measurement time
                nu = T.dscalar('nu')
                nu.tag.test_value = 1.
                
                N_AP, = T.nonzero(k_with_N_AP)
                p_N_AP_arr, updates = theano.scan(
                    fn = lambda N, gamma, delta, nu_max : integrateOut(p_nu(nu,gamma,delta,nu_max)*poisson_spikes(nu,N,T_total),nu,vectorize=False)(N,gamma,delta,nu_max), 
                    sequences=[N_AP],
                    non_sequences=[gamma,delta,nu_max],
                )
                k_eff = k_with_N_AP[N_AP]
                p_k = theano_binomial()(n,k_eff) * p_N_AP_arr**k_eff * (1-p_N_AP_arr)**(n - k_eff)
                #log_p_k = T.log(p_k)

                #return log_p_k
                T.printing.Print('N_AP (inside) ')(N_AP)
                T.printing.Print('k_with_AP (inside) ')(k_eff)
                T.printing.Print('logP (inside) ')(p_k)
                return T.switch(T.isnan(p_k), -100., T.log(p_k))

                #scaled_NU = tt.log(data / nu_max)
                #return - tt.log( nu_max / gamma * tt.sqrt( -np.pi * scaled_NU ) ) - delta**2 / 2 + \
                #    ( gamma**2 - 1 ) * scaled_NU + \
                #    tt.log( tt.cosh( gamma * delta * tt.sqrt( -2 * scaled_NU ) ) )

            # def logp_raw(data,paras):
            #     scaled_NU = np.log(data / nu_max)
            #     return - np.log( nu_max / gamma * np.sqrt( -np.pi * scaled_NU ) ) - delta**2 / 2 + \
            #         ( gamma**2 - 1 ) * scaled_NU + \
            #         np.log( np.cosh( gamma * delta * np.sqrt( -2 * scaled_NU ) ) )

        elif func=='lognorm':
            assert all(key in paras for key in ("mu","sigma")), "Please provide all necessary parameters!"

            def logp(data,paras):
                return 0


        self.logp = logp
        # self.logp_raw = logp_raw

        for key in paras.keys():
            self.paras[key] = {}
            for para in paras[key].keys():
                self.paras[key][para] = paras[key][para]


    def load_data(self, dataType='empirical', filePath='../data/BuscheLab/2P_data.xlsx',include_silent=False):

        self.mP = ModelParams(dataType, filePath=filePath, population_keys=['*mouse_type','animal'])
        self.data_df = self.mP.regularize_rates()

        self.data = self.data_df.to_numpy()

        N_zeros = (self.data==0).sum()
        print(f'zeros in data: {N_zeros}')
#        self.data[self.data==0] = np.random.rand(N_zeros)*1./600
        #if include_silent:
        #    T = 600.
        #    self.data[self.data<=1./T] = -np.log(1-np.random.rand((self.data<=1./T).sum()))/T

        self.data_mask = ~np.isnan(self.data) & (self.data>0)

        self.dataLoaded = True


    def construct_model_hierarchical(self, name):

        """
            function to create a hierarchical model of the parameter 'name'.

            Input:
                name: string
                    name of the parameter, used for distribution name
                mP: ModelParams Object
                    including 'rates' DataFrame, arranged in a multicolumnar structure
                    reflecting the different populations. Column names starting with '*'
                    are used on the hierarchical level to infer mean value lower level
                    distributions
                mu_population: float
                    mean value of top-level prior
                sigma_population: float
                    std of top-level prior
                sigma_animal:
                    std of low-level prior

            Output:
                prior_animal: pm.distribution
                    distribution containing a draw for each neuron in the dataset
                    ! this should be masked, when populations contain different neuron
                    numbers

            ToDo:
                * enable using different distributions, other than 'Normal' (default)
                * in both, top and bottom level?
                * what about Student-T?
        """

        # obtain information from data-column-header
        population_names = list(self.mP.rates.columns.names)
        N_levels = len(population_names)
        
        print(N_levels)

        # only columns with '*' are used for top-level inference
        is_hierarchical = [name.startswith('*') for name in population_names]
        
        print(f'name: {name}')
        print(self.paras[name])

        hierarchical_shape = [
            sha if is_hierarchical[p] else 1
            for p,sha in enumerate(self.mP.data_shape)
        ]
        print(f'hierarchical shape: {hierarchical_shape}')

        # create top-level distribution for prior mean-values
        prior_mean = self.paras[name]['mu'] + \
            pm.Normal(f'{name}_population',
                mu=0.,
                sigma=self.paras[name]['sigma'],
                shape=hierarchical_shape
            )
        tt.printing.Print('prior mean shape')(tt.shape(prior_mean))

        # create real distribution, from which values are drawn
        prior_animal = pm.Normal(f'{name}',
            mu=prior_mean,
            sigma=self.paras[name]['sigma_animal'],
            shape=self.mP.data_shape
        )
        prior_animal = prior_animal.flatten()
        tt.printing.Print('prior animal')(tt.shape(prior_animal))
        #prior_animal = prior_animal.reshape((1,-1))

        #tt.printing.Print('prior animal')(tt.shape(prior_animal))

        # broadcast distribution to draw values for each neuron
        #prior_animal = tt.tile(
        #    prior_animal,
        #    (self.data.shape[0], 1) ### why -1?    *[1]*(N_levels-1)
        #)
        #tt.printing.Print('prior animal (final)')(tt.shape(prior_animal))

        return prior_animal

    def prepare_data(self,T_total=100.):

        N_AP = self.data*T_total
        
        N_AP_max = int(np.nanmax(N_AP))+1
        data_observed = {'n_animals': [],
                         #'N_AP': np.zeros((N_AP_max,N_AP.shape[1]),dtype="int64"),
                         'k_animals': np.zeros((N_AP.shape[1],N_AP_max),dtype="int64")
                        }
        
        for a,N in enumerate(N_AP.T):
        
            N = N[~np.isnan(N)].astype('int64')
            N_ct = Counter(N)
            data_observed['k_animals'][a,list(N_ct.keys())] = list(N_ct.values())
        
            data_observed['n_animals'].append(len(N))
            
        print(f'data: {self.data}')
        print(f'data observed: {data_observed}')
        return data_observed
        
    
    ## currently issues: ~99% of traces diverge -> pole is difficult to fit
    def run_on_data(self,draws=5000,tune=10000,loadPath=None,savePath=None,**kwargs):

        """
            ToDo:
                * adjust logp method, such that it calculates
                    \int_0^1/T logp(nu) dnu
                to estimate probability of 0Hz measurements, instead of point-measure at 1/T
                -> use scipy.integrate.quad
                * write documentation!
        """

        if loadPath:
            self.trace = az.from_netcdf(loadPath)
            return

        #data_observed = self.data[self.data_mask]
        
        ## prepare data such that we have all necessary values
        
        data_observed = self.prepare_data()
        
        nu = T.dscalar('nu')
        nu.tag.test_value = 1.
        
        with pm.Model() as model:
            # replace normals with student-t distributions

            priors = {}
            for para in self.paras:
                priors[para] = self.construct_model_hierarchical(para)
                priors[para] = priors[para][self.mP.animal_mask]


            def likelihood(n_animals,k_animals):
                
                p_sum, updates = theano.scan(
                    fn = lambda n, k_with_N_AP, gamma, delta, nu_max : T.sum(self.logp(n,k_with_N_AP,gamma,delta,nu_max)), 
                    sequences=[n_animals,k_animals,priors['gamma'],priors['delta'],priors['nu_max']],
#                    non_sequences=[priors['gamma'],priors['delta'],priors['nu_max']],
                )                
                T.printing.Print('logP')(p_sum)

                #logP = tt.switch(tt.isnan(logP), min_val*2, logP)

                return T.sum(p_sum)

            ## watch out: for some reason, NaNs in observed data are converted to 0s
            logP = pm.DensityDist('logP',likelihood,observed=data_observed)

            trace = pm.sample(
                init='adapt_diag',
                step=pm.Metropolis(),
                chains=4,draws=draws,tune=tune,
                return_inferencedata=True,
                **kwargs)

            if savePath:
                trace.to_netcdf(savePath)

            self.trace = trace


In [11]:
I = Inference()
I.load_data('empirical',filePath='../../data/BuscheLab/2P_data.xlsx',include_silent=True)
I.set_model('selfcon')

column names: [['WT', '20180310A'], ['WT', '20180312A'], ['WT', '20180310B'], ['WT', '20180401'], ['LM (APLP1 KO)', '20180325 (app ko7 739)'], ['LM (APLP1 KO)', '20180325A (app ko8 740)'], ['LM (APLP1 KO)', '20180402A (app ko13 734)'], ['LM (APLP1 KO)', '20180419 (app ko14 756)'], ['cTKO', '20180309'], ['cTKO', '20180310'], ['cTKO', '20180311'], ['cTKO', '20180329'], ['cTKO', '20180329A']]
zeros in data: 1480


In [37]:
I.run_on_data(tune=200,draws=1000,loadPath=None)

data: [[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [       nan        nan        nan ... 6.92307692        nan        nan]
 [       nan        nan        nan ... 7.30769231        nan        nan]
 [       nan        nan        nan ... 8.07692308        nan        nan]]
data observed: {'n_animals': [256, 277, 355, 286, 331, 279, 171, 274, 310, 146, 370, 153, 247], 'k_animals': array([[ 71,   0,   7, ...,   0,   0,   0],
       [108,   0,   1, ...,   0,   1,   0],
       [110,   0,   0, ...,   0,   0,   0],
       ...,
       [227,   0,   9, ...,   0,   0,   0],
       [ 90,   0,   0, ...,   0,   0,   0],
       [181,   0,   0, ...,   0,   0,   0]])}
2
name: gamma
{'mu': 1.5, 'sigma': 1.0, 'sigma_animal': 1.0, 'prior': 'Normal'}
hierarchical shape: [3, 1]
prior mean shape __str__ = [3 1]
prior animal __str__ = 

/tmp/ipykernel_6529/4243496430.py:50: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  output_storage[0][0] = np.array(quad(f,0,nu_max,args=tuple(inputs))[0],dtype='float64')


N_AP (inside)  __str__ = [  0   2   3   5   6   7   8   9  11  13  14  15  16  17  19  20  22  23
  25  26  28  29  30  34  37  38  42  43  46  53  57  61  65  66  76  83
  84  92 100 107 111 119 157]
k_with_AP (inside)  __str__ = [71.  7. 30.  9.  2. 17. 14.  3. 18.  1.  5.  8.  3.  4.  7.  1.  4.  4.
  1.  7.  1.  1.  5.  4.  2.  4.  4.  1.  1.  1.  1.  1.  2.  1.  3.  1.
  1.  1.  1.  1.  1.  1.  1.]
logP (inside)  __str__ = [6.15024350e-022 4.76079528e-002 7.02437910e-008 1.21048867e-001
 2.17924175e-002 1.80259880e-004 1.60540553e-003 1.17907767e-001
 1.25526867e-006 6.47650570e-002 1.53681572e-001 2.10569136e-002
 2.16674800e-001 1.84885885e-001 2.21411061e-002 1.59009777e-001
 4.82854611e+267             inf            -inf            -inf
            -inf            -inf            -inf             inf
             inf             inf             inf            -inf
            -inf            -inf            -inf            -inf
             inf             nan             nan

KeyboardInterrupt: 

In [12]:
#theano.config.compute_test_value = "ignore"

def run_inference(filePath='../../data/BuscheLab/2P_data.xlsx',tune=20000,draws=10000,include_silent=False,
                 loadPath=None,savePath=None):
    I = Inference()
    I.load_data('empirical',filePath=filePath,include_silent=include_silent)
    I.set_model('selfcon')
    I.run_on_data(tune=tune,draws=draws,loadPath=loadPath)
    return I

I = run_inference(tune=1000,draws=1000,include_silent=True)#,loadPath='results_xls_noSilent.nc')

column names: [['WT', '20180310A'], ['WT', '20180312A'], ['WT', '20180310B'], ['WT', '20180401'], ['LM (APLP1 KO)', '20180325 (app ko7 739)'], ['LM (APLP1 KO)', '20180325A (app ko8 740)'], ['LM (APLP1 KO)', '20180402A (app ko13 734)'], ['LM (APLP1 KO)', '20180419 (app ko14 756)'], ['cTKO', '20180309'], ['cTKO', '20180310'], ['cTKO', '20180311'], ['cTKO', '20180329'], ['cTKO', '20180329A']]
zeros in data: 1480
data: [[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [       nan        nan        nan ... 6.92307692        nan        nan]
 [       nan        nan        nan ... 7.30769231        nan        nan]
 [       nan        nan        nan ... 8.07692308        nan        nan]]
data observed: {'n_animals': [256, 277, 355, 286, 331, 279, 171, 274, 310, 146, 370, 153, 247], 'k_animals': array([[ 71, 108, 110, ..., 

AttributeError: 'CVM' object has no attribute 'maker'

In [12]:
data_observed = I.prepare_data(T_total=10.)
data_observed['k_animals'].shape
#N_AP = [  0   2   3   5   6   7   8   9  11  13  14  15  16  17  19  20  22  23
#  25  26  28  29  30  34  37  38  42  43  46  53  57  61  65  66  76  83
#  84  92 100 107 111 119 157]
#k_with_AP (inside)  __str__ = [71.  7. 30.  9.  2. 17. 14.  3. 18.  1.  5.  8.  3.  4.  7.  1.  4.  4.
#  1.  7.  1.  1.  5.  4.  2.  4.  4.  1.  1.  1.  1.  1.  2.  1.  3.  1.
#  1.  1.  1.  1.  1.  1.  1.]

data: [[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [       nan        nan        nan ... 6.92307692        nan        nan]
 [       nan        nan        nan ... 7.30769231        nan        nan]
 [       nan        nan        nan ... 8.07692308        nan        nan]]
data observed: {'n_animals': [256, 277, 355, 286, 331, 279, 171, 274, 310, 146, 370, 153, 247], 'k_animals': array([[ 71,   0,   7, ...,   0,   0,   0],
       [108,   0,   1, ...,   0,   1,   0],
       [110,   0,   0, ...,   0,   0,   0],
       ...,
       [227,   0,   9, ...,   0,   0,   0],
       [ 90,   0,   0, ...,   0,   0,   0],
       [181,   0,   0, ...,   0,   0,   0]])}


(13, 197)

In [14]:
### define theano constants and variables
T_total = theano.shared(10.)                     # measurement time

n_animals = T.dvector('n_animals')                 # number of neurons in data set
k_animals = T.dmatrix('k_animals')

n = T.dscalar('n')
k_with_N_AP = T.dvector('k_with_N_AP')   # number of neurons observed with N_AP spikes

N_AP = T.dvector('N_AP')           # number of action potentials (should be vector)
#N = T.dscalar('N')

nu = T.dvector('nu')

gamma = T.dscalar('gamma')
delta = T.dscalar('delta')
nu_max = T.dscalar('nu_max')

In [89]:
class integrateOut(Op):
    """
    Integrate out a variable from an expression, computing
    the definite integral w.r.t. the variable specified
    
    !!! vectorized implementation still somewhat buggy in the gradient !!!
    
    function adapted from https://stackoverflow.com/questions/42678490/custom-theano-op-to-do-numerical-integration
    """
    
    def __init__(self,f,t,vectorize,*args,**kwargs):
        super(integrateOut,self).__init__()
        self.f = f
        self.t = t
        self.vectorize = vectorize

    def make_node(self,*inputs):
        self.fvars=list(inputs)
        # This will fail when taking the gradient... don't be concerned
        try:
            self.gradF = T.jacobian(self.f,self.fvars)
        except:
            self.gradF = None
            
        if self.vectorize:
            return Apply(self,self.fvars,[T.dcol().type()])
        else:
            return Apply(self,self.fvars,[T.dscalar().type()])

    def perform(self,node, inputs, output_storage):
        nu_max = inputs[3]
        
        ## integrate the function from 0 to maximum firing rate nu_max
        if self.vectorize:
            f = theano.function([self.t]+self.fvars,self.f)
            
            def g(x):
                return f(x,tuple(inputs))
            #def p_nu():
            #    return = gamma / ( nu_max * np.sqrt( -np.pi * np.log( nu / nu_max ) ) ) * \
            #        np.exp( - delta**2/2.) * ( nu / nu_max )**(gamma**2 - 1) * \
            #        np.cosh( gamma * delta * np.sqrt( -2 * np.log( nu / nu_max) ) )
            
            #def poisson(nu,k,T):
            #    return (nu*T)**k / sp_factorial(k) * np.exp(-nu*T)
            #N_AP = inputs[0]
            #output = np.zeros_like(N_AP,dtype='float64')
            #for i,N in enumerate(N_AP):
            #    args = inputs[:]
            #    args[0] = np.array([N])   # necessary to be 1-dim vector to satisfy function-blueprint
            #    #print(f'args: {args}')
            #    output[i] = quad(f,0,nu_max,args=tuple(args))[0]
            #output_storage[0][0] = output
            
            print(f'f = {g([[1],[2],[3],[4]])}')
            
            ### implement incremental increase of eps upon fail
            #output = quadpy.quad(f,0,nu_max,args=tuple(inputs),epsabs=1e-6, epsrel=1e-6, limit=100)[0]
            output = quadpy.c1.integrate_adaptive(g,[[0,nu_max]],eps_abs=1e-6, eps_rel=1e-6, max_num_subintervals=100)
            print(output)
            output_storage[0][0] = output
            
        else:
            f = theano.function([self.t]+self.fvars,self.f)
            #print(f'N: {inputs[0]}')
            #print(f'nu_max: {nu_max}, gamma: {inputs[1]}, delta: {inputs[2]}')
            #T.printing.Print('nu_max')(nu_max)
            output_storage[0][0] = np.array(quad(f,0,nu_max,args=tuple(inputs))[0],dtype='float64')

    def grad(self,inputs,output_grads):
        nu_max = inputs[3]
        
        # need to map variable names of function to input variables, 
        # as they might not agree (scan creates differently named copies)
        giv = {}
        for v,v_new in zip(self.fvars[1:],list(inputs)[1:]):
            giv[v] = v_new
        
        if self.vectorize:
            return [T.mean(integrateOut(theano.clone(g,replace=giv),self.t,self.vectorize)(*inputs))*output_grads[0] \
                for g in self.gradF]
        else:
            return [integrateOut(theano.clone(g,replace=giv),self.t,self.vectorize)(*inputs)*output_grads[0] \
                for g in self.gradF]

In [77]:
class theano_poisson(Op):
    """
    specific theano implementation of poisson distribution to allow efficient 
    calculus of array-valued k inputs
    """
    
    def __init__(self,T_total,*args,**kwargs):
        super(theano_poisson,self).__init__()
        self.T = T_total

    def make_node(self,*inputs):
        self.fvars=list(inputs)
            
        return Apply(self,self.fvars,[T.dmatrix().type()])

    def perform(self,node, inputs, output_storage):
        nu,k = inputs
        ## broadcast variables manually, as they are created dynamically
        #nu = nu[:,np.newaxis]
        k = k[:,np.newaxis]
        output = (nu*self.T)**k / sp_factorial(k) * np.exp(-nu*self.T)
        output_storage[0][0] = np.array(output,dtype='float64')

In [59]:
N_AP, = T.nonzero(k_with_N_AP)
poiss = theano_poisson(10.)(nu,N_AP)

poiss_fun = theano.function([nu,k_with_N_AP],poiss)
poiss_fun([1.,1.5,2.],data_observed['k_animals'][0,:])

[1.  1.5 2. ]
[  0   2   3   5   6   7   8   9  11  13  14  15  16  17  19  20  22  23
  25  26  28  29  30  34  37  38  42  43  46  53  57  61  65  66  76  83
  84  92 100 107 111 119 157]


array([[4.53999298e-005, 2.26999649e-003, 7.56665496e-003,
        3.78332748e-002, 6.30554580e-002, 9.00792257e-002,
        1.12599032e-001, 1.25110036e-001, 1.13736396e-001,
        7.29079462e-002, 5.20771044e-002, 3.47180696e-002,
        2.16987935e-002, 1.27639962e-002, 3.73216263e-003,
        1.86608131e-003, 4.03913704e-004, 1.75614654e-004,
        2.92691090e-005, 1.12573496e-005, 1.48906741e-006,
        5.13471521e-007, 1.71157174e-007, 1.53776714e-009,
        3.29851382e-011, 8.68029952e-012, 3.23129766e-014,
        7.51464571e-015, 8.25059916e-017, 1.06201566e-021,
        1.12024132e-024, 8.94438525e-028, 5.50458937e-031,
        8.34028692e-032, 2.40785242e-040, 1.15066922e-046,
        1.36984431e-047, 3.64997737e-055, 4.86464918e-063,
        3.70152319e-070, 2.57522131e-074, 8.14409029e-083,
        3.87055403e-126],
       [3.05902321e-007, 3.44140111e-005, 1.72070055e-004,
        1.93578812e-003, 4.83947030e-003, 1.03702935e-002,
        1.94443003e-002, 3.240

In [60]:
import quadpy
theano.config.compute_test_value = 'off'
def logp(n,k_with_N_AP,gamma,delta,nu_max):

    if n==0:
        return 0
    T.printing.Print('k:')(k_with_N_AP)
    N_AP, = T.nonzero(k_with_N_AP)
    T.printing.Print('N_AP:')(N_AP)
    #p_N_AP_arr = p_nu(nu,gamma,delta,nu_max) * theano_poisson(10.)(nu,N_AP)#
    p_N_AP_arr = integrateOut(p_nu(nu,gamma,delta,nu_max)*theano_poisson(10.)(nu,N_AP),nu,vectorize=True)(N_AP,gamma,delta,nu_max)
    #p_N_AP_arr, updates = theano.scan(
    #    fn = lambda N, gamma, delta, nu_max : integrateOut(p_nu(nu,gamma,delta,nu_max)*poisson_spikes(nu,N,T_total),nu,vectorize=True)(N,gamma,delta,nu_max), 
    #    sequences=[N_AP],
    #    non_sequences=[gamma,delta,nu_max],
    #)
    #k_eff = k_with_N_AP[N_AP]
    #p_k = theano_binomial()(n,k_eff) * p_N_AP_arr**k_eff * (1-p_N_AP_arr)**(n - k_eff)
    #log_p_k = T.log(p_k)
    return p_N_AP_arr
    #return p_k
    #return T.switch(T.isnan(log_p_k), -100., log_p_k)

#p_sum, updates = theano.scan(
#    fn = lambda n, k_with_N_AP, gamma, delta, nu_max : T.sum(logp(n,k_with_N_AP,gamma,delta,nu_max)), 
#    sequences=[n_animals,k_animals],
#    non_sequences=[gamma,delta,nu_max],
#)

In [90]:
import quadpy

nu = T.drow('nu')
k_with_N_AP = T.dvector('k_with_N_AP')

T_total = theano.shared(10.)                     # measurement time

gamma = T.dscalar('gamma')
delta = T.dscalar('delta')
nu_max = T.dscalar('nu_max')

N_AP, = T.nonzero(k_with_N_AP)

p_dist = p_nu(nu,gamma,delta,nu_max)
p_poiss = theano_poisson(10.)(nu,N_AP)

p_joint = p_dist * p_poiss

p_N_AP_arr = integrateOut(p_dist * p_poiss,nu,vectorize=True)(N_AP,gamma,delta,nu_max)

#p_N_AP_arr = integrateOut(p_nu(nu,gamma,delta,nu_max)*theano_poisson(10.)(nu,N_AP),nu,vectorize=True)(N_AP,gamma,delta,nu_max)

p_dist_fun = theano.function([nu,gamma,delta,nu_max],p_dist)
p_poiss_fun = theano.function([nu,k_with_N_AP],p_poiss)
p_joint_fun = theano.function([nu,k_with_N_AP,gamma,delta,nu_max],p_joint)
p_N_AP_fun = theano.function([k_with_N_AP,gamma,delta,nu_max],p_N_AP_arr)

nu_ = np.array([[1,2,3]])
p_dist_val = p_dist_fun(nu_,1.2,4.8,30.)
p_poiss_val = p_poiss_fun(nu_,data_observed['k_animals'][0,:])
p_joint_val = p_joint_fun(nu_,data_observed['k_animals'][0,:],1.2,4.8,30.)
p_N_AP_val = p_N_AP_fun(data_observed['k_animals'][0,:],1.2,4.8,30.)

print(f'p dist = {p_dist_val}')
print(f'p poiss = {p_poiss_val}')
print(f'p joint = {p_joint_val}')
print(f'p N_AP = {p_N_AP_val}')

TypeError: can only concatenate str (not "tuple") to str
Apply node that caused the error: <__main__.integrateOut object at 0x7f1817d48070>(Nonzero.0, gamma, delta, nu_max)
Toposort index: 1
Inputs types: [TensorType(int64, vector), TensorType(float64, scalar), TensorType(float64, scalar), TensorType(float64, scalar)]
Inputs shapes: [(43,), (), (), ()]
Inputs strides: [(8,), (), (), ()]
Inputs values: ['not shown', array(1.2), array(4.8), array(30.)]
Inputs type_num: [7, 12, 12, 12]
Outputs clients: [['output']]

Backtrace when the node is created(use Theano flag traceback__limit=N to make it longer):
  File "/home/wollex/anaconda3/envs/inference/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 2936, in _run_cell
    return runner(coro)
  File "/home/wollex/anaconda3/envs/inference/lib/python3.9/site-packages/IPython/core/async_helpers.py", line 129, in _pseudo_sync_runner
    coro.send(None)
  File "/home/wollex/anaconda3/envs/inference/lib/python3.9/site-packages/I

In [64]:
p_dist_val[:,np.newaxis] * p_poiss_val

array([[2.06559630e-006, 1.03279815e-004, 3.44266049e-004,
        1.72133025e-003, 2.86888374e-003, 4.09840535e-003,
        5.12300668e-003, 5.69222965e-003, 5.17475423e-003,
        3.31715015e-003, 2.36939296e-003, 1.57959531e-003,
        9.87247067e-004, 5.80733569e-004, 1.69805137e-004,
        8.49025685e-005, 1.83771793e-005, 7.99007797e-006,
        1.33167966e-006, 5.12184486e-007, 6.77492706e-008,
        2.33618174e-008, 7.78727248e-009, 6.99650006e-011,
        1.50075076e-012, 3.94934411e-013, 1.47016890e-015,
        3.41899744e-016, 3.75383996e-018, 4.83193612e-023,
        5.09685000e-026, 4.06949727e-029, 2.50446630e-032,
        3.79464591e-033, 1.09551954e-041, 5.23529019e-048,
        6.23248833e-049, 1.66065890e-056, 2.21330768e-064,
        1.68411111e-071, 1.17166868e-075, 3.70538078e-084,
        1.76101639e-127],
       [2.82741040e-011, 5.65482081e-009, 3.76988054e-008,
        7.53976108e-007, 2.51325369e-006, 7.18072484e-006,
        1.79518121e-005, 3.989

In [202]:
k_test = T.lrow('k_test')
_,N_AP = T.nonzero(k_test)

test_fun = theano.function([k_test],N_AP)
test_fun(data_observed['k_animals'][0:1,:])


array([  0,   2,   3,   5,   6,   7,   8,   9,  11,  13,  14,  15,  16,
        17,  19,  20,  22,  23,  25,  26,  28,  29,  30,  34,  37,  38,
        42,  43,  46,  53,  57,  61,  65,  66,  76,  83,  84,  92, 100,
       107, 111, 119, 157])

In [58]:
import quadpy
def p_nu_(nu,gamma,delta,nu_max):
    return gamma / ( nu_max * np.sqrt( -np.pi * np.log( nu / nu_max ) ) ) * \
        np.exp( - delta**2/2.) * ( nu / nu_max )**(gamma**2 - 1) * \
        np.cosh( gamma * delta * np.sqrt( -2 * np.log( nu / nu_max) ) )

def poisson_spikes_(nu,N,T_total):
    #T.gamma(N+1)
    return (nu*T_total)**N[:,np.newaxis] / sp_factorial(N,exact=True)[:,np.newaxis].astype('float64') * np.exp(-nu*T_total)

def p_N_AP(nu,N,gamma,delta,nu_max,T_total=10.):
    return p_nu_(nu[np.newaxis,:],gamma,delta,nu_max) * poisson_spikes_(nu[np.newaxis,:],N,T_total)

def g(x):
    return p_N_AP(x,np.arange(50),1.2,4.8,30.,T_total=10.)
quadpy.c1.integrate_adaptive(g,[[0],[10]],eps_abs=1e-6, eps_rel=1e-6, max_num_subintervals=100)[0]

array([[7.01884082e-01, 1.07426470e-01, 4.76088464e-02, 2.80764521e-02,
        1.88931911e-02, 1.37236868e-02, 1.04809780e-02, 8.29384663e-03,
        6.73970242e-03, 5.59100651e-03, 4.71541768e-03, 4.03124721e-03,
        3.48562873e-03, 3.04300273e-03, 2.67867071e-03, 2.37500610e-03,
        2.11913352e-03, 1.90145678e-03, 1.71469644e-03, 1.55324391e-03,
        1.41271768e-03, 1.28965216e-03, 1.18127544e-03, 1.08534789e-03,
        1.00004327e-03, 9.23859780e-04, 8.55552926e-04, 7.94084043e-04,
        7.38580551e-04, 6.88304928e-04, 6.42630267e-04, 6.01020870e-04,
        5.63016722e-04, 5.28220968e-04, 4.96289760e-04, 4.66923965e-04,
        4.39862344e-04, 4.14875934e-04, 3.91763366e-04, 3.70346967e-04,
        3.50469494e-04, 3.31991374e-04, 3.14788375e-04, 2.98749633e-04,
        2.83775960e-04, 2.69778401e-04, 2.56677000e-04, 2.44399726e-04,
        2.32881555e-04, 2.22063667e-04]])

In [51]:
N_ = np.arange(100,200)
sp_factorial(N_,exact=True)

array([...],
      dtype=object)

In [73]:
gamma = 1.2
delta = 4.8
nu_max = 300.
nu_bar = get_nu_bar(gamma,delta,nu_max)
alpha_0 = get_alpha_0(gamma,delta,nu_max)
tau_I = get_tau_I(nu_max,tau_m=0.01)
print(f'nu_bar: {nu_bar}, alpha_0: {alpha_0}, tau_I: {tau_I}')

nu_bar: 2.0519561063000684, alpha_0: nan, tau_I: 2.8144773233982723e-05


/home/wollex/Data/Science/WolfGroup/DarkMatter/Programme/DM_theory/functions.py:33: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(J_0**2 * ( nu_mean/ (2 * gamma**2 * (tau_I + tau_m)) - q))


In [ ]:
psum_fun = theano.function([n_animals,k_animals,gamma,delta,nu_max],p_sum)
psum_fun(data_observed['n'],data_observed['k_with_N_AP'],1.2,4.8,30.)

In [ ]:
psum_fun(data_observed['n'],data_observed['k_with_N_AP'],1.6,4.8,30.)

In [ ]:
pGrad = T.jacobian(logp(n,k_with_N_AP,gamma,delta,nu_max),[gamma,delta,nu_max],consider_constant=[n,k_with_N_AP])
funcGrad = theano.function([n,k_with_N_AP,gamma,delta,nu_max],pGrad)
funcGrad(data_observed['n'][0],data_observed['k_with_N_AP'][0,:],1.6,4.8,30.)

In [ ]:
pGrad = T.jacobian(p_sum,[gamma,delta,nu_max],consider_constant=[n_animals,k_animals])
funcGrad = theano.function([n_animals,k_animals,gamma,delta,nu_max],pGrad)
funcGrad(data_observed['n'],data_observed['k_with_N_AP'],1.6,4.8,30.)

In [ ]:
p_N_AP_arr, updates = theano.scan(
    fn = lambda N, gamma, delta, nu_max : integrateOut(p_nu(nu,gamma,delta,nu_max)*poisson_spikes(nu,N,T_total),nu,vectorize=False)(N,gamma,delta,nu_max), 
    sequences=[N_AP],
    non_sequences=[gamma,delta,nu_max],
)
p_k = theano_binomial()(n,k_with_N_AP) * p_N_AP_arr**k_with_N_AP * (1-p_N_AP_arr)**(n - k_with_N_AP)
func_p = theano.function([N_AP,gamma,delta,nu_max],p_N_AP_arr)
func_vals = func_p([0,3,5,10],1.2,4.8,30.)
print(f'val = {func_vals}')

pGrad = T.jacobian(p_N_AP_arr,[gamma,delta,nu_max],consider_constant=[N_AP])
funcGrad = theano.function([N_AP,gamma,delta,nu_max],pGrad)

In [ ]:
grad_vals = funcGrad([0,3,5,10],1.2,4.8,30.)
print(f'grad = {grad_vals}')

In [ ]:
#p_N_AP = integrateOut(p_nu(nu,gamma,delta,nu_max)*poisson_spikes(nu,N,T_total),nu)(N,gamma,delta,nu_max)

#p_N_AP_arr, updates = theano.scan(
#    fn = lambda N, gamma, delta, nu_max : integrateOut(p_nu(nu,gamma,delta,nu_max)*poisson_spikes(nu,N,T_total),nu)(gamma,delta,nu_max,N), 
#    sequences=[N_AP],
#    non_sequences=[gamma,delta,nu_max]
#)
p_k = theano_binomial()(n,k_with_N_AP) * p_N_AP_arr**k_with_N_AP * (1-p_N_AP_arr)**(n - k_with_N_AP)

#p_N_AP_val = theano.function([N_AP,gamma,delta,nu_max], p_N_AP_arr, updates=updates)
#res = p_N_AP_val([0,5,10,50],1.2,4.8,30.)
#print(f'p_N_AP: {res}')

p_k_val = theano.function(inputs=[N_AP,k_with_N_AP,n,gamma,delta,nu_max], outputs=p_k)
res = p_k_val([0,5,10,50],[5,5,5,5],300,1.2,4.8,30.)
print(f'p_k = {res}')

In [ ]:

pGrad = T.jacobian(p_k,[gamma,delta,nu_max])
funcGrad = theano.function([N_AP,k_with_N_AP,n,gamma,delta,nu_max],pGrad)
grad_vals = funcGrad([0,5,10,50],[5,5,5,5],300,1.2,4.8,30.)


#pGrad = T.jacobian(p_N_AP_arr,[gamma,delta,nu_max])
#print('---')
#funcGrad = theano.function([N_AP,gamma,delta,nu_max],pGrad) ### somehow in here, copies are made, which dont fit ..
## ... the original definition of variables, as they have different names...
## WHY??

#print('---')
#grad_vals = funcGrad([0,3,5,10],1.2,4.8,30.)
print(grad_vals)

In [ ]:
distr = p_nu(nu,gamma,delta,nu_max)*poisson_spikes(nu,N,T_total)
jac = T.jacobian(distr,[gamma,delta,nu_max])
j_val = theano.function(inputs=[nu,gamma,delta,nu_max,N],outputs=jac)
j_val(1.,1.2,4.8,30.,0)

In [ ]:
def get_val_and_grad(f,T_vals,vals):
    func = theano.function(T_vals,f)
    val = func(*vals)
    
    #try:
    #grad = T.grad(f,T_vals)
    grad = T.jacobian(f,T_vals)
    funcGrad = theano.function(T_vals,grad)
    grad_vals = funcGrad(*vals)
    print(f"value @provided input: {val},\
          \ngradient: {grad_vals}")
    #except:
    #    print(f"value @provided input: {val},\
    #       \ngradient not implemented")

In [ ]:
rng = np.random.RandomState(42)
def p_N_AP_fun(gamma,delta,nu_max,N_AP):
    #p_N_AP = integrateOut(p_nu(nu,gamma,delta,nu_max)*poisson_spikes(nu,N,T_total),nu)(gamma,delta,nu_max,N)
    
    p_N_AP, updates = theano.scan(
        fn = lambda N : integrateOut(p_nu(nu,gamma,delta,nu_max)*poisson_spikes(nu,N,T_total),nu)(gamma,delta,nu_max,N), 
        sequences=[N_AP],
    )
    
    return p_N_AP
theano.gradient.verify_grad(p_N_AP_fun,[1.1,3.2,50.,[0.,3.,5.]],rng=rng,mode='DebugMode')

In [ ]:
get_val_and_grad(p_N_AP_arr,[gamma,delta,nu_max,N_AP],[1.2,4.8,30.,[0,5]])

In [ ]:
#from theano import tests
rng = np.random.RandomState(42)
mode = 'DebugMode'

#p_test_ = theano.function([gamma,delta,nu_max],p_N_AP_spikes)
#theano.gradient.verify_grad(p_test_, [1.2,4.8,30.],
theano.gradient.verify_grad(binomial(N_total,k_with_N_AP), [np.asarray(0.05)], 
#theano.gradient.verify_grad(integrateOut(distribution_N_AP_spikes,nu,0.,10.), [np.asarray(1.2),np.asarray(4.8),np.asarray(30.)], 
                            rng=rng, 
                            mode=mode, no_debug_ref=False)